In [3]:
import datetime
from datetime import datetime, timedelta
import calendar
import requests
import json
import urllib3
import configparser
import csv
config = configparser.ConfigParser()
config.read('credentials.ini')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
MDM_add = config['API']['MDM_add']
MDM_user = config['API']['MDM_username']
MDM_password = config['API']['MDM_password']

In [4]:
def get_devices(add, user, password):
    url= f"{add}/api/1/devices/"
    try:
        r = requests.get(url, auth=(MDM_user, MDM_password), verify=False)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return []
    
    device = r.json()
    
    del_key = ['communicationId', 'typeId', 'typeName', 'templateId', 'templateName', 'managementState', 'description', 'manufacturer', 'model', 'parentId', 'location', 'storeData', 'groupId']
    for item in device:
        for k in del_key:
            item.pop(k, None)
    return device

In [1]:
dev_list=get_devices(MDM_add, MDM_user, MDM_password)

NameError: name 'get_devices' is not defined

In [10]:
print(dev_list[0].get('id'))

ISKIE077058


In [11]:
def billing_time_data(month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    # Use current year and month if not provided
    now = datetime.now()
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    
    month = month.capitalize()
    cur_start_date = datetime(year, list(calendar.month_name).index(month), 
                              calendar.monthrange(year, list(calendar.month_name).index(month))[1],
                              start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")        
    return start_time_cur, end_time_cur


def daily_time_data(day: int = None, month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    now = datetime.now()    
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    month = month.capitalize()    
    if day is None:
        day = now.day
        month = now.strftime('%B')
        year = now.year
    else:
        date_obj = datetime(year, list(calendar.month_name).index(month), day) - timedelta(days=1)
        year, month, day = date_obj.year, date_obj.strftime('%B'), date_obj.day
    cur_start_date = datetime(year, list(calendar.month_name).index(month), day, start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")

    return start_time_cur, end_time_cur


while True:
    profile = input("Enter the profile you want to extract data for (Billing or Daily): ").strip().lower()
    if profile == "billing":
        prof = "1-0:98.1.0*255"
        tprof = "Billing Profile"
        while True:
            try:
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                start_time_cur, to_time_cur = billing_time_data(month, year)
                break
            except (ValueError, IndexError):
                print("Invalid date. Please try again.")
        break
    elif profile == "daily":
        prof = "1-0:99.2.0*255"
        tprof = "Daily Profile"
        while True:
            try:
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                day_input = input("Enter the day (e.g., 31) or press Enter to use today's date: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                day = int(day_input) if day_input else None
                start_time_cur, to_time_cur = daily_time_data(day, month, year)
                break
            except (ValueError, IndexError):
                print("Invalid Date. Please try again")
        break
            
    else:
        print("Invalid choice. Please enter either 'billing' or 'daily'.")
        
end_date_part = start_time_cur.split("T")[0]


Enter the profile you want to extract data for (Billing or Daily):  Billing
Enter the month (e.g., January) or press Enter to use the current month:  June
Enter the year (e.g., 2024) or press Enter to use the current year:  


In [14]:
end_date_part = start_time_cur.split("T")[0]

print("Data Will Be Extracted For Following: ")
print("======================================")
#print(f"Data Source   : {data_source}")
print(f"Profile       : {tprof}")
print(f"Profile Value : {prof}")
print(f"Start Time    : {start_time_cur}")
print(f"End Time      : {to_time_cur}")

Data Will Be Extracted For Following: 
Profile       : Billing Profile
Profile Value : 1-0:98.1.0*255
Start Time    : 2024-06-30T18:29:00Z
End Time      : 2024-06-30T18:31:00Z


In [41]:
schema=[]
#print(dev_list[0])
for devices in dev_list:
    sch = {}
    sch["device"] = devices.get('id')
    sch["profile"] = prof
    sch["from"] = start_time_cur
    sch["to"] = to_time_cur
    schema.append(sch)    
print(len(schema))

9995


In [43]:
def split_schema(biglist, shard_size):
    # Using list comprehension to create sublists of the given size
    return [biglist[i:i + shard_size] for i in range(0, len(biglist), shard_size)]


In [44]:
# Split the schema into sublists
sublists = split_schema(schema, 3000)
#num_sublists = len(sublists)
#sublist_sizes = [len(sublist) for sublist in sublists]

#print(f"Number of sublists created: {num_sublists}")
#print(f"Sizes of each sublist: {sublist_sizes}")


Number of sublists created: 4
Sizes of each sublist: [3000, 3000, 3000, 995]


In [52]:
request_url = f"{MDM_add}/api/1/bulk/device-profiles/metered-data/get"
try:
    r = requests.post(url, auth=(MDM_user, MDM_password), verify=False, json=sublists[0])
except requests.exceptions.RequestException as e:
    print(f"Request Error: {e}")

val = r.json()



In [58]:
print(len(val))
print(val[0])

3000
{'success': True, 'value': [{'capturedAt': '2024-06-30T18:30:00Z', 'readingReason': 1, 'statusWord': 0, 'dataSource': 64, 'meteredValues': [{'registerId': '0-0:0.1.2*255', 'value': 1719772200, 'unit': 's', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:13.0.0*255', 'value': 0.991, 'unit': '1', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.0*255', 'value': 520.167, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.1*255', 'value': 75.047, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.2*255', 'value': 72.782, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.3*255', 'value': 60.654, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.4*255', 'value': 106.47, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.5*255', 'value': 205.214, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:9.8.0*255

In [66]:
for item in val:
    item.pop("success")

In [67]:
for i in range(0,2):
    print(val[i])

{'value': [{'capturedAt': '2024-06-30T18:30:00Z', 'readingReason': 1, 'statusWord': 0, 'dataSource': 64, 'meteredValues': [{'registerId': '0-0:0.1.2*255', 'value': 1719772200, 'unit': 's', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:13.0.0*255', 'value': 0.991, 'unit': '1', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.0*255', 'value': 520.167, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.1*255', 'value': 75.047, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.2*255', 'value': 72.782, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.3*255', 'value': 60.654, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.4*255', 'value': 106.47, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:1.8.5*255', 'value': 205.214, 'unit': 'kWh', 'measuredAt': '2024-06-30T18:30:00Z'}, {'registerId': '1-0:9.8.0*255', 'value': 607.679, '

In [ ]:
def get_profile_data(address, username, password, profile, devlist, from_time, to_time):
    m_values_kwh = []
    m_values_kvah = []
    for device in devlist:
        device_id = device.get('id', 'N/A')
        groupName = device.get('groupName', 'N/A')
        url = f"{address}/api/1/devices/{device_id}/profiles/{profile}/entries?from={from_time}&to={to_time}"
        try:
            r = requests.get(url, auth=(username, password), verify=False)
            r.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            continue
        response_dict = r.json()
        entries = response_dict.get("entries", [])
        if entries:
            first_entry = entries[0]
            capturedAt = first_entry.get("capturedAt", [])
            metered_value = first_entry.get("meteredValues", [])

            for item in metered_value:
                if item["registerId"] == "1-0:1.8.0*255":
                    item["device"] = device_id
                    item["profile"] = profile
                    item["groupName"] = groupName
                    if item["measuredAt"] is None:
                        item["measuredAt"] = capturedAt
                    m_values_kwh.append(item)
                elif item["registerId"] == "1-0:9.8.0*255":
                    item["device"] = device_id
                    item["profile"] = profile
                    item["groupName"] = groupName
                    if item["measuredAt"] is None:
                        item["measuredAt"] = capturedAt
                    m_values_kvah.append(item)
    return m_values_kwh, m_values_kvah